# Exercise 1 - Hazard Data

Generate hazard module from source data

In this exercise, we will extract data from Hurdat (https://www.nhc.noaa.gov/data/) and convert the data into an Oasis compliant hazard module format.




### Import python libraries

In [ ]:
# standard python libraries
import requests
import re
import os
from math import sin, cos, sqrt, atan2, radians

# non-standard python libraries
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

from shapely.geometry import Point
from geopandas import GeoDataFrame

# constants

#import geopy.distance
#km_nm_factor = 1.852
#earth_radius = 6373.0


### Download event data from hurdat

In [ ]:
# get data from URL and write to file
raw_fn = 'source_data/raw_hurdat_data.txt'
url = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2020-052921.txt'
data = requests.get(url)
with open(raw_fn,'w') as fw:
    fw.write(data.text)
    
# create an empty list to temoporarily store downloaded data
data=[]

# loop through downloaded data and parse by row
with open(raw_fn,'r') as fr:
    for row in fr:
        if row[0:2]=='AL':
            event_id=row[0:8]
            event_name=row[19:28]
            records=row[34:36]
        else:
            date=row[0:8]
            time=row[10:14]
            record_id=row[16]
            system_status=row[19:21]
            latitude=row[23:27]
            lat_hem=row[27]
            longitude=row[30:35]
            lon_hem=row[35]
            max_windspeed=row[39:41]
            min_pressure=row[43:47]
            radii_34kt_ne=row[49:53]
            radii_34kt_se=row[55:59]
            radii_34kt_sw=row[61:65]
            radii_34kt_nw=row[67:71]
            radii_50kt_ne=row[73:77]
            radii_50kt_se=row[79:83]
            radii_50kt_sw=row[85:89]
            radii_50kt_nw=row[91:95]
            radii_64kt_ne=row[97:101]
            radii_64kt_se=row[103:107]
            radii_64kt_sw=row[109:113]
            radii_64kt_nw=row[115:119]
            
            row_data = [
                event_id,
                event_name,
                records,
                date,
                time,
                record_id,
                system_status,
                latitude,
                lat_hem,
                longitude,
                lon_hem,
                max_windspeed,
                min_pressure,
                radii_34kt_ne,
                radii_34kt_se,
                radii_34kt_sw,
                radii_34kt_nw,
                radii_50kt_ne,
                radii_50kt_se,
                radii_50kt_sw,
                radii_50kt_nw,
                radii_64kt_ne,
                radii_64kt_se,
                radii_64kt_sw,
                radii_64kt_nw
            ]
            data.append(row_data)
    
# create a pandas dataframe with the list data
cols = ['id','name','records','date','time','record_id','system_status','latitude',
        'lat_hem','longitude','lon_hem','max_windspeed','min_pressure',
        'radii_34kt_ne','radii_34kt_se','radii_34kt_sw','radii_34kt_nw',
        'radii_50kt_ne','radii_50kt_se','radii_50kt_sw','radii_50kt_nw',
        'radii_64kt_ne','radii_64kt_se','radii_64kt_sw','radii_64kt_nw']



df_data=pd.DataFrame(data=data,columns=cols)

dtypes = {'id':str,'name':str,'records':int,'date':int,'time':int,
          'record_id':str,'system_status':str,'latitude':float,'lat_hem':str,
          'longitude':float,'lon_hem':str,'max_windspeed':int,'min_pressure':int,
          'radii_34kt_ne':int,'radii_34kt_se':int,'radii_34kt_sw':int,'radii_34kt_nw':int,
          'radii_50kt_ne':int,'radii_50kt_se':int,'radii_50kt_sw':int,'radii_50kt_nw':int,
          'radii_64kt_ne':int,'radii_64kt_se':int,'radii_64kt_sw':int,'radii_64kt_nw':int}

df_data = df_data.astype(dtypes)

# strip leading spaces
df_data['name']=df_data['name'].str.strip()

# set negative longitudes for western hemisphere
df_data['longitude']=-df_data['longitude']

# write the dataframe out to csv
formatted_fn = 'source_data/formatted_hurdat_data.csv'
df_data.to_csv(formatted_fn,index=False)

df_data

### Inspect the data for Hurricane HARVEY

In [ ]:
# if you don't have access to the hurdat website, the data can be found here
# df_data = pd.read_csv('source_data/formatted_hurdat_data.csv')

df_data[df_data['id']=='AL092017'].head(20)

## Get model area grid

In [ ]:
# get events which make happened since 2000
df_data_in_period = df_data[df_data['date']>=20000101]

df_data_in_period

## Get a list of distinct events in the remaining event set

this is used in Oasis to batch the exection job

In [ ]:
# combine data
df_events = df_data_in_period['id'].drop_duplicates().reset_index()[['id']]
df_events['event_id']=df_events.index+1

df_events

In [ ]:
# add the event ids into the data dataframe
df_data_in_period = df_data_in_period.merge(df_events,on='id')

# Create a uniform grid for the US

This is used to discretise the hazard data over area

In [ ]:
df_grid = pd.read_csv('source_data/us_grid.csv')
df_grid

### display the grid on a map

In [ ]:
# show grid
geometry = [Point(xy) for xy in zip(df_grid['longitude'], df_grid['latitude'])]
gdf = GeoDataFrame(df_grid, geometry=geometry)   

#this is a simple map that comes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(20, 12)), marker='o', color='red', markersize=1);

### create the cartesian indicies (x,y) for the grid and add in area peril id

In [ ]:
# create cartisian indicies
df_x = df_grid[['latitude']].drop_duplicates().reset_index()[['latitude']]
df_x['x_index']=df_x.index+1

df_y = df_grid[['longitude']].drop_duplicates().reset_index()[['longitude']]
df_y['y_index']=df_y.index+1

df_grid_coord = df_grid.merge(df_x,on='latitude').merge(df_y,on='longitude')

# create area peril id for grid
df_grid_coord['area_peril_id']=df_grid_coord.index+1

del df_grid_coord['geometry']

df_grid_coord

In [ ]:
# find areaperil cell of track point
df_events_ap = df_data_in_period.merge(df_grid_coord,on=['latitude','longitude'])

df_events_ap

### remove hazard records below a windspeed threshold

we don't need low windspeed records, as they don't cause damage

In [ ]:
# remove records below windspeed threhold
v_thresh = 45
df_events_thresh = df_events_ap[df_events_ap['max_windspeed'] >= v_thresh]

df_events_thresh

### display remaining events

In [ ]:
df_events_thresh[['id','name']].drop_duplicates()

### index the intensity values

In [ ]:
# index windspeeds and assign size
df_intensity = df_events_thresh['max_windspeed'].drop_duplicates().sort_values().reset_index()
df_intensity['intensity_bin_index']=df_intensity.index+1

df_intensity = df_intensity[['intensity_bin_index','max_windspeed']]

df_intensity

In [ ]:
df_events_intensity = df_events_thresh.merge(df_intensity,on='max_windspeed').sort_values(by=['date','time'])
df_events_intensity

### map the area peril ids to the footprint

In [ ]:
def get_ap(x,y):
    print(x,y)
    area_peril_id = df_grid_coord[(df_grid_coord['x_index']==x) &
                    (df_grid_coord['y_index']==y)]['area_peril_id'].values[0]
    
    return area_peril_id

x=55
y=197

try:
    ap = df_grid_coord[(df_grid_coord['x_index']==x) &
                    (df_grid_coord['y_index']==y)]['area_peril_id'].values[0]
    print(ap)
except:
    pass

In [ ]:
df_events_intensity = df_events_thresh.merge(df_intensity,on='max_windspeed').sort_values(by=['date','time'])

# generate intensity values per event & areaperil


lst_fp = []

def get_ap(x,y):
    try:
        area_peril_id = df_grid_coord[(df_grid_coord['x_index']==x) &
                    (df_grid_coord['y_index']==y)]['area_peril_id'].values[0]
    except:
        area_peril_id = 0
    return area_peril_id
    
for index, row in df_events_intensity.iterrows():
    event_id = row['event_id']
    area_peril_id = row['area_peril_id']
    intensity_bin = row['intensity_bin_index']
        
    row_fp = [event_id,area_peril_id,intensity_bin]
    lst_fp.append(row_fp)
    
    # assume "radius" = intensity bin
    radius = intensity_bin
    if radius > 1:
        x0 = df_grid_coord[df_grid_coord['area_peril_id']==area_peril_id]['x_index'].values[0]
        y0 = df_grid_coord[df_grid_coord['area_peril_id']==area_peril_id]['y_index'].values[0]
        for r in range(radius-1):
            intensity_bin = intensity_bin - 1
            x0 = x0-1
            y0 = y0+1
            area_peril_id = get_ap(x0,y0)
            row_fp = [event_id,area_peril_id,intensity_bin]
            lst_fp.append(row_fp)
            for c in range(2*r-1):
                x0=x0+1
                area_peril_id = get_ap(x0,y0)
                if area_peril_id > 0:
                    row_fp = [event_id,area_peril_id,intensity_bin]
                    lst_fp.append(row_fp)
            for c in range(2*r-1):
                y0=y0-1
                area_peril_id = get_ap(x0,y0)
                if area_peril_id > 0:
                    row_fp = [event_id,area_peril_id,intensity_bin]
                    lst_fp.append(row_fp)
            for c in range(2*r-1):
                x0=x0-1
                area_peril_id = get_ap(x0,y0)
                if area_peril_id > 0:
                    row_fp = [event_id,area_peril_id,intensity_bin]
                    lst_fp.append(row_fp)
            for c in range(2*r-1-1):
                y0=y0+1
                area_peril_id = get_ap(x0,y0)
                if area_peril_id > 0:
                    row_fp = [event_id,area_peril_id,intensity_bin]
                    lst_fp.append(row_fp)                
    
df_footprint = pd.DataFrame(data=lst_fp,columns=['event_id','area_peril_id','intensity_bin_index'],dtype='int').sort_values(
    by=['event_id','area_peril_id','intensity_bin_index'])
df_footprint['probability']=1

df_footprint
    

In [ ]:
# write model files out
df_events[['event_id','id']].to_csv('model_data/events.csv',index=False)
df_footprint[df_footprint['area_peril_id']>0].to_csv('model_data/footprint.csv',index=False)
df_intensity.to_csv('model_data/intensity_bin_dict.csv',index=False)

### save the area peril grid for use in mapping exposure (later)

In [ ]:
# write keys data out
df_grid_coord[['area_peril_id','latitude','longitude']].to_csv('keys_data/areaperil_dict.csv',index=False)

### generate occurrence file

this is used in oasis to map events to years

In [ ]:
# write occurrence file
df_event_dates = df_events_intensity.groupby(['id'])[['date']].min().merge(df_events,on='id')
df_event_dates['date_str']=df_event_dates['date'].astype('str')
df_event_dates['occ_year']=df_event_dates['date_str'].apply(lambda x: int(x[0:4]))
df_event_dates['occ_month']=df_event_dates['date_str'].apply(lambda x: int(x[4:6]))
df_event_dates['occ_day']=df_event_dates['date_str'].apply(lambda x: int(x[6:8]))

year_min = df_event_dates['occ_year'].min()
df_event_dates['period_no']=df_event_dates['occ_year'] - year_min + 1

df_occurrence = df_event_dates[['event_id','period_no','occ_year','occ_month','occ_day']].sort_values(
    by=['period_no','event_id'])

df_occurrence.to_csv('model_data/occurrence.csv',index=False)

df_occurrence

### generate a return periods file

this defines the return periods which we want to report on in the output EP curves

In [ ]:
# write return period file
with open('model_data/returnperiods.csv','w') as rp:
    rp.write('20\n10\n5\n3\n2\n1')
             
! cat model_data/returnperiods.csv